In [ ]:
# Python中引用式变量，创建对象之后才会吧变量分配给对象
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles # 别名
print(lewis is charles)
print(id(lewis), id(charles))
lewis['balance'] = 950
print(charles) # 增加了balance

alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
print(alex == charles) # == 比较两个对象的值
print(alex is charles) # is比较对象的标识

# 在变量和单例值之间比较时，应该使用is
# x is None
# x is not None
# is 运算符比 == 快，==是语法糖，等同于a.__eq__(b)

# 元组是不可变的，但元素依然可变
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
print(t1 == t2)
print(id(t1[-1]), id(t2[-1]))
t1[-1].append(50)
print(t1 == t2)
print(id(t1[-1]), id(t2[-1]))
# t1最后一个元素的内容发生了改变

# 默认浅复制 - 可变序列
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l3 = l1[:]  # 浅赋值简便写法
print(l2 == l1)
print(l3 == l1)

l1.append(100)  # 对l2无影响
l1[1].remove(55) # 影响l2，浅复制时l1和l2的元素指向同一个列表的引用
print('l1 = %s' % l1)
print('l2 = %s' % l2)
l2[1] += [33, 22]  # 对列表而言，+= 在原列表中增加元素
l2[2] += (10, 11)  # 对元组而言，+= 创建新元组而后赋值
print(l1[1] == l2[1])
print('l1 = %s' % l1)
print('l2 = %s' % l2)


In [ ]:
import copy

class Bus:

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)  # 浅拷贝 bus2与bus1共用passengers列表
bus3 = copy.deepcopy(bus1) # 深拷贝 bus3与bus1的passengers列表独立
bus1.drop('Bill')
print(bus2.passengers)
print(bus3.passengers)

# 通过实现__copy__ __deepcopy__方法，可以控制copy和deepcopy的行为。

# 深复制会记住已经复制的对象，能优雅地处理循环引用。
a = [10, 20]
b = [a, 30]
a.append(b)
print(a)
print(copy.deepcopy(a))

# ***Python中唯一支持的参数传递模式时共享传参(call by sharing)，各形式参数获得实参中各个引用的副本***
# 函数可能修改作为参数传入的可变对象，但是无法修改那些对象的标识（即不能把一个对象替换成另一个对象）

In [ ]:
# 不要使用可变类型作为参数的默认值
class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):  # <1>
        self.passengers = passengers  # <2>

    def pick(self, name):
        self.passengers.append(name)  # <3>

    def drop(self, name):
        self.passengers.remove(name)

bus1 = HauntedBus(['Alice', 'Bill']) # bus1用指定的列表初始化
bus1.pick('Charlie')
bus1.drop('Alice')
print('bus1 : %s' % bus1.passengers)

bus2 = HauntedBus() # bus2用缺省列表初始化，之后所有的缺省参数均使用相同的一个列表
bus2.pick('Carrie')
print('bus2 : %s' % bus2.passengers)
bus3 = HauntedBus() # bus3用缺省列表，实际和bus2共用一个
print('bus3 : %s' % bus3.passengers)
bus3.pick('Dave')
print('bus3 : %s' % bus3.passengers)
print('bus1 : %s' % bus1.passengers)
print(bus2.passengers is bus3.passengers)
print(HauntedBus.__init__.__defaults__) # 可以看到共用的缺省参数内容
print(HauntedBus.__init__.__defaults__[0] is bus2.passengers) # bus2.passengers其实是default[0]的别名

In [ ]:
# 防范可变参数 对可变参数的变更可能会波及不期望的范围
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']

class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []  # 缺省时创建一个新的空列表
        else:
            self.passengers = passengers  # 使用指定的列表

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)  # append和remove会修改列表的内容

bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
print('bus : %s' % bus.passengers)
print('team : %s' % basketball_team) # 下车后的学生不在篮球队了，变更被扩散到不期望的范围

basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []  # 缺省时创建一个新的空列表
        else:
            self.passengers = list(passengers)  # 使用指定的列表的副本，限制影响范围

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)  # append和remove会修改列表的内容

bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
print('bus : %s' % bus.passengers)
print('team : %s' % basketball_team) # team 不受影响

In [ ]:
# del删除名称而不是对象，但当删除的变量保存的是对象最后一个引用或者无法得到对象时（
# 如两个对象循环引用，垃圾回收程序会判定它们都无法获取），对象被当作垃圾回收
# 重新绑定也可能会导致对象的引用数量归零，导致对象被销毁。

import weakref

s1 = {1, 2, 3}
s2 = s1
def bye():
    print('Gone with the wind....')

ender = weakref.finalize(s1,bye) # 在s1引用的对象上注册bye回调
print(ender.alive)
del s1 # 减引用计数
print(ender.alive)
s2 = 'spam' # 重新绑定，导致原对象的计数归零，bye被调用
print(ender.alive)

In [ ]:
# 弱引用
# 弱引用常用作缓存中，不会增加对象的引用计数，不会让对象存在的时间超过所需时间。
# 引用的目标对象称为所指对象referent
import weakref

a_set = {0, 1}
wref = weakref.ref(a_set) # 创建弱引用对象
print(wref)
print(wref()) # 所指对象
a_set = {2, 3, 4}  # 重新绑定对象
print(wref()) # 不存在
print(wref() is None)


In [ ]:

# weakref.ref类其实是底层接口，一般程序应使用WeakKeyDictionary、WeakValueDictionary、WeakSet和finalize
class Cheese:

    def __init__(self, kind):
        self.kind = kind

    def __repr__(self):
        return 'Cheese(%r)' % self.kind

stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]
for cheese in catalog:
    stock[cheese.kind] = cheese # 弱引用

print(sorted(stock.keys())) # 全部列表
del catalog
print(sorted(stock.keys())) # 删除列表后，各个Cheese对象的引用归零，处理for循环中临时变量使用的那个
del cheese
print(sorted(stock.keys())) # 删除临时变量后，最后一个Cheese对象的引用归零

# list和dict实例不能作为所指对象，但它们的子类可以。